## Set system path

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/DLC-Jupyter-Notebooks
sys.path.append(str(project_root))


## Connect to database

In [ ]:
import os
import psycopg2
import pandas as pd
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.49"
conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password="1234", host=host, port="5432")
cursor = conn.cursor()


## Insert normalized (x,y) of bodyparts

In [ ]:
# First import the submodules as objects so they can be reloaded
import Python_scripts.Extract_db_columns.find_csv_for_video as find_mod
import Python_scripts.Extract_db_columns.normalize_bodypart_from_csv as from_mod
import Python_scripts.Extract_db_columns.normalize_bodypart_minmax as minmax_mod
import Python_scripts.Extract_db_columns.normalize_bodypart_with_fallback as fallback_mod

# Reload them
importlib.reload(find_mod)
importlib.reload(from_mod)
importlib.reload(minmax_mod)
importlib.reload(fallback_mod)

# Now import the functions freshly from the reloaded modules
from Python_scripts.Extract_db_columns.find_csv_for_video import find_csv_for_video
from Python_scripts.Extract_db_columns.normalize_bodypart_from_csv import normalize_bodypart_from_csv
from Python_scripts.Extract_db_columns.normalize_bodypart_minmax import normalize_bodypart_minmax
from Python_scripts.Extract_db_columns.normalize_bodypart_with_fallback import normalize_bodypart_with_fallback


In [ ]:
df_raw = pd.read_csv('/Users/atanugiri/Downloads/data/ToyLight/DlcDataPytorchFiltered/ToyLight_10_16_23_BlackCement_PDLC_Resnet50_DLC-FoodLightJul8shuffle1_snapshot_240_filtered.csv', header=[1, 2], index_col=0)
print(df_raw.columns[:10])
print(df_raw[('Corner1', 'likelihood')].head())

In [ ]:
query = """
SELECT id, video_name
FROM dlc_table
WHERE task ILIKE 'LightOnly%' AND genotype='white' AND frame_rate IS NOT NULL ORDER BY id;
"""
df = pd.read_sql_query(query, conn)


In [ ]:
from tqdm import tqdm
import numpy as np

for _, row in tqdm(df.iterrows(), total=len(df), desc="Inserting normalized head"):
    video_id = row["id"]
    video_name = row["video_name"]
    dir_to_search = Path.home() / "Downloads" / "data" / "LightOnlyWhite" / "DlcDataPytorchFiltered"

    csv_path = find_csv_for_video(video_name, dir_to_search)

    if not csv_path:
        print(f"❌ CSV not found for {video_name}")
        continue

    df_dlc = pd.read_csv(csv_path, header=[1, 2], index_col=0)

    # Subset to required columns
    needed_parts = ['Head'] + [f'Corner{i}' for i in range(1, 5)]
    coords = ['x', 'y', 'likelihood']

    print(f"Normalizing id = {video_id}")
    
    x_norm, y_norm = normalize_bodypart_minmax(
        df_dlc, bodypart='Head', likelihood_threshold=0.5)

    if x_norm is None:
        print(f"❌ Normalization failed for video_id {video_id}")
        continue

    # Insert normalized
    cursor = conn.cursor()
    cursor.execute("""
        ALTER TABLE dlc_table 
        ADD COLUMN IF NOT EXISTS head_x_norm FLOAT[],
        ADD COLUMN IF NOT EXISTS head_y_norm FLOAT[];
    """)
    cursor.execute("""
        UPDATE dlc_table
        SET head_x_norm = %s, head_y_norm = %s
        WHERE id = %s;
    """, (np.round(x_norm, 3).tolist(), np.round(y_norm, 3).tolist(), video_id))
    conn.commit()
    cursor.close()
    print(f"✅ Normalized head for ID {video_id}\n")


## Plot a few trajectories side by side

In [ ]:
query = """
SELECT id FROM dlc_table WHERE task ILIKE 'LightOnly%' AND genotype = 'white'
AND head_x_norm IS NOT NULL ORDER BY id
"""

df = pd.read_sql_query(query, conn)
id_list = df['id'].tolist()
print(id_list)

from Python_scripts.Data_analysis.plot_single_trajectory import plot_single_trajectory
from Python_scripts.Data_analysis.plot_single_trajectory_from_csv import plot_single_trajectory_from_csv

import matplotlib.pyplot as plt

dir_to_search = Path.home() / "Downloads" / "data" / "LightOnlyWhite" / "DlcDataPytorchFiltered"

for id in id_list:
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    plot_single_trajectory_from_csv(
        conn, trial_id=id, base_data_dir=dir_to_search, 
        bodyparts=['Head'], style='scatter', color_by_time=True,
        likelihood_threshold=0.5, ax=axs[0])

    plot_single_trajectory(conn, id, bodypart_x='head_x_norm', bodypart_y='head_y_norm', 
                           label=f"{id}", style='line', color_by_time=True, ax=axs[1])
    
    plt.tight_layout()
    plt.show()


In [ ]:
print(len(id_list))


In [ ]:
query = "SELECT id FROM dlc_table WHERE task='ToyLight' AND health='ghrelin' AND head_x_norm IS NOT NULL ORDER BY id"

df = pd.read_sql_query(query, conn)
id_list = df['id'].tolist()
# print(id_list)


from Python_scripts.Data_analysis.plot_single_trajectory import plot_single_trajectory
from Python_scripts.Data_analysis.plot_single_trajectory_from_csv import plot_single_trajectory_from_csv

import matplotlib.pyplot as plt

for id in id_list:
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    plot_single_trajectory_from_csv(
        conn, trial_id=id, bodyparts=['Head'], style='scatter', color_by_time=False,
        likelihood_threshold=0.5, ax=axs[0])

    plot_single_trajectory(conn, id, bodypart_x='head_x_norm', bodypart_y='head_y_norm', 
                           label=f"{id}", style='line', color_by_time=True, ax=axs[1])
    
    plt.tight_layout()
    plt.show()


## Compare DLC vs Optogen

In [ ]:
import importlib
import Python_scripts.Data_analysis.plot_single_trajectory
import Python_scripts.Data_analysis.plot_single_trajectory_optogen

importlib.reload(Python_scripts.Data_analysis.plot_single_trajectory)
importlib.reload(Python_scripts.Data_analysis.plot_single_trajectory_optogen)

from Python_scripts.Data_analysis.plot_single_trajectory import plot_single_trajectory
from Python_scripts.Data_analysis.plot_single_trajectory_optogen import plot_single_trajectory_optogen

query = "SELECT id FROM dlc_table WHERE video_name LIKE '%FoodLight_9_10_24_Indigo_Y%'"

df = pd.read_sql_query(query, conn)
id = df['id'][0]
print(id)
print(type(id))
plot_single_trajectory(conn, id, max_points=500)
plot_single_trajectory_optogen(conn, id, max_points=500)


In [ ]:
import os
import shutil
from pathlib import Path

# Source and destination directories
src_dir = Path("/Users/atanugiri/Downloads/data/FoodLightWhite/DlcDataPytorch")
dst_dir = Path("/Users/atanugiri/Downloads/data/FoodLightWhite/DlcDataPytorchFiltered")

# Create destination directory if it doesn't exist
dst_dir.mkdir(parents=True, exist_ok=True)

# Process all .csv files
for csv_file in src_dir.glob("*.csv"):
    base = csv_file.stem  # filename without .csv extension
    new_name = f"{base}_filtered.csv"
    dst_path = dst_dir / new_name

    shutil.copy(csv_file, dst_path)
    print(f"Copied and renamed: {csv_file.name} -> {new_name}")
